In [1]:
#########################################################################################
# Funciones
#########################################################################################
  
import cv2 
import numpy as np
import os
import glob # libreria para buscar los archivos a procesar
import torch 
from tqdm import tqdm


def frames_to_video(frames_folder, output_path, fps=30):
  """
  Crea un video a partir de una carpeta con los frames del video.

  Args:
    frames_folder: La ruta a la carpeta que contiene los frames del video.
    output_path: La ruta al archivo de video de salida.
    fps: La cantidad de frames por segundo del video.
  """

  # Obtener la lista de frames en la carpeta
  frames = sorted([f for f in os.listdir(frames_folder) if f.endswith(('.jpg', '.png'))])

  # Leer el primer frame para obtener las dimensiones del video
  first_frame = cv2.imread(os.path.join(frames_folder, frames[0]))
  height, width, _ = first_frame.shape

  # Crear el objeto VideoWriter
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Códec para MP4
  video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

  # Escribir cada frame en el video
  for frame_name in frames:
    frame_path = os.path.join(frames_folder, frame_name)
    frame = cv2.imread(frame_path)
    video_writer.write(frame)

  # Liberar recursos
  video_writer.release()
  cv2.destroyAllWindows()



def recorte_auto(input_video_path, output_folder, output_video_path, umbral):
    """
    Procesa un video de una piscina de metal fundido en movimiento horizontal,
    detecta los límites vertical y horizontal donde aparece la piscina entre el
    primer y último frame, recorta todos los frames a estos límites y 
    genera un nuevo video.
    
    Args:
        input_video_path: Ruta del video de entrada
        output_folder: Carpeta donde guardar los frames recortados
        output_video_path: Ruta donde guardar el video resultante
        umbral: Valor de pixel que sirve como umbral entre 0 y 255 
    """
    # Crear carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Abrir el video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: No se pudo abrir el video {input_video_path}")
        return
    
    # Obtener propiedades del video
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Procesando video: {frame_count} frames, {width}x{height} píxeles, {fps} FPS")
    
    # Variables para almacenar los límites verticales
    min_y = height
    max_y = 0
    
    # Variables para almacenar los límites horizontales
    # Inicializamos con valores que garantizan que cubran toda la anchura inicialmente
    min_x_global = width
    max_x_global = 0
    
    # Primera pasada: Detectar contornos en todos los frames para determinar límites verticales
    # print("Pasada 1: Detectando límites verticales...")
    
    for _ in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convertir a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Aplicar umbral fijo de 60
        _, thresh = cv2.threshold(gray, umbral, 255, cv2.THRESH_BINARY)
        
        # Detectar contornos
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            # Elegir el contorno más grande (asumimos que es la piscina)
            c = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(c)
            
            # Actualizar los límites verticales globales
            min_y = min(min_y, y)
            max_y = max(max_y, y + h)
    
    # Reiniciar el video para detectar los límites horizontales en el primer y último frame
    # print("Detectando límites horizontales en el primer y último frame...")
    cap.release()
    cap = cv2.VideoCapture(input_video_path)
    
    # Obtener primer frame
    ret, first_frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el primer frame")
        return
    
    # Procesar primer frame para obtener límites horizontales
    gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, umbral, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)
        min_x_first = x
        max_x_first = x + w
        # print(f"Límites horizontales en el primer frame: x_min={min_x_first}, x_max={max_x_first}")
    else:
        min_x_first = 0
        max_x_first = width
    
    # Posicionar en el último frame
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)
    
    # Obtener último frame
    ret, last_frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el último frame")
        return
    
    # Procesar último frame para obtener límites horizontales
    gray = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, umbral, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)
        min_x_last = x
        max_x_last = x + w
        # print(f"Límites horizontales en el último frame: x_min={min_x_last}, x_max={max_x_last}")
    else:
        min_x_last = 0
        max_x_last = width
    
    # Determinar los límites horizontales globales
    # Extendemos la región para que incluya la posición de la piscina tanto en el inicio como en el final
    min_x_global = min(min_x_first, min_x_last)
    max_x_global = max(max_x_first, max_x_last)
    
    # Añadir un margen de seguridad
    margin = 3
    min_y = max(0, min_y - margin)
    max_y = min(height - 1, max_y + margin)
    min_x_global = max(0, min_x_global - margin)
    max_x_global = min(width - 1, max_x_global + margin)
    
    # print(f"Límites finales para recorte: y_min={min_y}, y_max={max_y}, x_min={min_x_global}, x_max={max_x_global}")
    
    # Reiniciar el video para la tercera pasada (recorte y guardado)
    cap.release()
    cap = cv2.VideoCapture(input_video_path)
    
    # Tercera pasada: recortar y guardar los frames
    # print("Pasada 3: Recortando frames y guardando...")
    
    # Dimensiones del frame recortado
    new_height = max_y - min_y + 1
    new_width = max_x_global - min_x_global + 1
    
    # Preparar el escritor de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (new_width, new_height))
    
    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recortar el frame según los límites detectados
        cropped_frame = frame[min_y:max_y+1, min_x_global:max_x_global+1]

        # Redimensionar para mantener una salida de tamaño constante
        #cropped_frame = cv2.resize(cropped_frame, (width, height))
        
        # Guardar el frame recortado
        output_path = os.path.join(output_folder, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(output_path, cropped_frame)
        
        # Añadir al video de salida
        out.write(cropped_frame)
        
        frame_number += 1
    
    # Liberar recursos
    cap.release()
    out.release()
    
    print(f"Recorte completado.\n")
    # print(f"Dimensiones del recorte: {new_width}x{new_height} píxeles")
    # print(f"Video guardado en: {output_video_path}")
    # print(f"Frames guardados en: {output_folder}")


def umbral(input_folder, output_folder, umbral):
    """
    Aplica filtro de umbralización para eliminar el ruido en las regiones fuera de la piscina

    Args:
    input_folder: ruta a carpeta con frames a procesar (string)
    output_folder: ruta a carpeta con frames procesados (string)
    umbral: valor de umbral entre 0 (negro) y 255 (blanco) (entero)
    """
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Obtener la lista de archivos en la carpeta de entrada
    # Se asume que las imágenes pueden tener extensión jpg, png, etc.
    image_paths = glob.glob(os.path.join(input_folder, "*.*"))
    
    for img_path in image_paths:
        # Leer la imagen en escala de grises
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"No se pudo cargar la imagen: {img_path}")
            continue
    
        # Aplicar el umbral TOZERO
        # Los pixeles con valor menor a umbral2 se pondrán a 0 y los mayores se mantendrán
        _, thresh_img = cv2.threshold(img, umbral, 255, cv2.THRESH_TOZERO)
    
        # Generar la ruta de salida usando el mismo nombre de archivo
        filename = os.path.basename(img_path)
        output_path = os.path.join(output_folder, filename)
    
        # Guardar la imagen procesada
        cv2.imwrite(output_path, thresh_img)
        #print(f"Procesada y guardada: {output_path}")
    
    print("Threshold completado.")

def CLAHE(input_folder, output_folder, tg, cl):
    """
    Aplica ecualizaciones por bloques (kernel) y une los resultados 
    de cada bloque en una sola imagen con interpolacion en los bordes.

    Args:
    input_folder: ruta a carpeta con frames a procesar (string)
    output_folder: ruta a carpeta con frames procesados (string)
    tg: Tile Grid. Tamaño del kernel cuadrado de tg x tg (entero)
    cl: Clip LImit. 
    """
    # input_folder = "resultados_threshold"
    # output_folder = f"resultados_CLAHE_{tg_cont}_{cl_cont}"
        
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    # Crear objeto CLAHE con parámetros (clipLimit y tileGridSize pueden ajustarse)
    clahe = cv2.createCLAHE(clipLimit=cl, tileGridSize=(tg, tg))
        
    # Obtener la lista de archivos de imagen en la carpeta de entrada
    image_paths = glob.glob(os.path.join(input_folder, "*.*"))
        
    # Procesar cada imagen
    for img_path in image_paths:
        # Cargar la imagen
        img = cv2.imread(img_path)
        if img is None:
            print(f"No se pudo cargar la imagen: {img_path}")
            continue
        
        # Convertir la imagen a escala de grises (CLAHE trabaja con imágenes en grises)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Aplicar CLAHE
        processed = clahe.apply(gray)
        
        # Generar la ruta de salida manteniendo el mismo nombre de archivo
        filename = os.path.basename(img_path)
        output_path = os.path.join(output_folder, filename)
        
        # Guardar la imagen procesada
        cv2.imwrite(output_path, processed)
        #print(f"Procesada y guardada: {output_path}")
        
    print("CLAHE completado.\n")
    

In [2]:
import itertools

tg = {2, 4, 8, 16, 32}
cl = {1, 2, 3, 4, 5}


for tg_cont, cl_cont in itertools.product(tg, cl):

    !rm -rf resultados_CLAHE/*
    !rm -rf resultados_threshold_post/*
    !rm -rf results/*

    input_folder = "resultados_threshold_pre"
    output_folder_CLAHE = "resultados_CLAHE"
    output_folder_threshold = "resultados_threshold_post"

    
    CLAHE(input_folder, output_folder_CLAHE, tg_cont, cl_cont)
    umbral(output_folder_CLAHE, output_folder_threshold, 30)
    !python inference_realesrgan.py -n RealESRGAN_x4plus -i resultados_threshold_post -dn 0 --fp32 --tile 200 --gpu-id 0

    frames_to_video("results", f"videos/sr_{tg_cont}_{cl_cont}.mp4")


CLAHE completado.

Threshold completado.
Testing 0 frame_0000
	Tile 1/1
Testing 1 frame_0001
	Tile 1/1
Testing 2 frame_0002
	Tile 1/1
Testing 3 frame_0003
	Tile 1/1
Testing 4 frame_0004
	Tile 1/1
Testing 5 frame_0005
	Tile 1/1
Testing 6 frame_0006
	Tile 1/1
Testing 7 frame_0007
	Tile 1/1
Testing 8 frame_0008
	Tile 1/1
Testing 9 frame_0009
	Tile 1/1
Testing 10 frame_0010
	Tile 1/1
Testing 11 frame_0011
	Tile 1/1
Testing 12 frame_0012
	Tile 1/1
Testing 13 frame_0013
	Tile 1/1
Testing 14 frame_0014
	Tile 1/1
Testing 15 frame_0015
	Tile 1/1
Testing 16 frame_0016
	Tile 1/1
Testing 17 frame_0017
	Tile 1/1
Testing 18 frame_0018
	Tile 1/1
Testing 19 frame_0019
	Tile 1/1
Testing 20 frame_0020
	Tile 1/1
Testing 21 frame_0021
	Tile 1/1
Testing 22 frame_0022
	Tile 1/1
Testing 23 frame_0023
	Tile 1/1
Testing 24 frame_0024
	Tile 1/1
Testing 25 frame_0025
	Tile 1/1
Testing 26 frame_0026
	Tile 1/1
Testing 27 frame_0027
	Tile 1/1
Testing 28 frame_0028
	Tile 1/1
Testing 29 frame_0029
	Tile 1/1
Testing 3